In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import subprocess

In [4]:
train_results_dir = os.getcwd() + "/../002-Training-models/train_results/"
# !ls {train_results_dir}

In [5]:
!mkdir trained_on_v2
%cd trained_on_v2
!mkdir logs

mkdir: trained_on_v1: File exists
/Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models/trained_on_v1


In [17]:
eval_results_dir = os.getcwd()
dataset_dir = os.getcwd()+"/../../datasets/"
output_file_name = "eval_trained_on_dataset_v2"
output_file = f"logs/{output_file_name}.txt"

In [8]:
# Download YOLOv8
!pip install ultralytics

from IPython import display
display.clear_output()

%cd {eval_results_dir}
!mkdir yolov8

from ultralytics import YOLO
print('YOLOv8 setup complete.')

/Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models/trained_on_v1
YOLOv8 setup complete.


In [12]:
# Download YOLOv7
%cd {eval_results_dir}
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!pip install -r requirements.txt

from IPython import display
display.clear_output()
print('YOLOv7 setup complete.')

YOLOv7 setup complete.


In [13]:
# Download YOLOv5
%cd {eval_results_dir}
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5

!pip install -qr requirements.txt  
import torch

from IPython import display
display.clear_output()
print('YOLOv5 setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

YOLOv5 setup complete. Using torch 2.0.0 CPU


In [18]:
%cd {eval_results_dir}

/Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models/trained_on_v1


# Evaluate vs original datasets

In [ ]:
total = len(os.listdir(train_results_dir))
counter = 0
for folder_name in sorted(os.listdir(train_results_dir)):
    counter += 1
    if not folder_name.startswith("model"):
        continue

    folder_path = os.path.join(train_results_dir, folder_name, "weights")
    print(f"{counter}/{total}")

    # Split the folder name into variables
    variables = folder_name.split('_')

    # Extract the required variables
    model = variables[1]
    tf = variables[3]
    imgsz = variables[5]
    epochs = variables[7]
    batch = variables[9]
    dataset = variables[11]
    loss = variables[13]
    lr = variables[15]
    
    print(folder_name)
    print("Model:", model)
    print("TF:", tf)
    print("Imgsz:", imgsz)
    print("Epochs:", epochs)
    print("Batch:", batch)
    print("Dataset:", dataset)
    print("Executing command...")
    print()
    
    dataset_location = dataset_dir + model + "_original_v2"
        
    command = None
    
    if model == "yolov5" and dataset == "v2":
        best_pt_path = os.path.join(folder_path, "best.pt")
        if os.path.exists(best_pt_path):
            %cd yolov5
            command = f"python3 val.py --weights {best_pt_path} --data {dataset_location}/data.yaml --task test --name {folder_name}"
            print(command)

    elif model == "yolov7" and dataset == "v2":
        best_pt_path = os.path.join(folder_path, "best.pt")
        if os.path.exists(best_pt_path):
            %cd yolov7
            command = f"python3 test.py --weights {best_pt_path} --data {dataset_location}/data.yaml --task test --name {folder_name}"
            print(command)

    elif model == "yolov8" and dataset == "v2":
        best_pt_path = os.path.join(folder_path, "best.pt")
        if os.path.exists(best_pt_path):
            %cd yolov8
            command = f"yolo val model={best_pt_path} data={dataset_location}/data.yaml split=test name={folder_name}"
            print(command)

    if command:
        # Run the command using subprocess.Popen and capture the output
        with subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True) as process:
            # Open the output file for writing
            with open(f"{eval_results_dir}/{output_file}", "a") as file:
                file.write(folder_name)
                file.write(f"\nModel: {model}")
                file.write(f"\nTF: {tf}")
                file.write(f"\nImgsz: {imgsz}")
                file.write(f"\nEpochs: {epochs}")
                file.write(f"\nBatch: {batch}")
                file.write(f"\nDataset: {dataset}")
                file.write(f"\nLoss: {loss}")
                file.write(f"\nLr0: {lr}\n")
                # Read and display each line of the output as it becomes available
                for line in process.stdout:
                    print(line, end="")
                    file.write(line)
        %cd ..

    from IPython import display
    display.clear_output()
    

print("Executing commands...", "DONE!")
print()

2/28
model_yolov5_tf_no_imgsz_800_epochs_100_batch_16_dataset_v1_loss_SGD_lr_01
Model: yolov5
TF: no
Imgsz: 800
Epochs: 100
Batch: 16
Dataset: v1
Executing command...

[Errno 2] No such file or directory: 'yolov5'
/Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models/trained_on_v1/yolov5
python3 val.py --weights /Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models/../002-Training-models/train_results/model_yolov5_tf_no_imgsz_800_epochs_100_batch_16_dataset_v1_loss_SGD_lr_01/weights/best.pt --data /Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models/trained_on_v1/../../datasets/yolov5_original/data.yaml --task test --name model_yolov5_tf_no_imgsz_800_epochs_100_batch_16_dataset_v1_loss_SGD_lr_01
val: data=/Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models/trained_on_v1/../../datasets/yolov5_original/data.yaml, weights=['/Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models/../002-Tra

## Convert the logs to CSV

In [ ]:
!mkdir {eval_results_dir}/output_csvs

In [ ]:
def process_file(file_path):
    results = list()
    curr_res = dict()

    with open(file_path, 'r') as file:
        lines = file.readlines()

    for line in lines:
        if line.startswith("Model:"):
            curr_res["model"] = line.split()[1]
        elif line.startswith("TF:"):
            curr_res["tf"] = line.split()[1]
        elif line.startswith("Imgsz:"):
            curr_res["imgsz"] = line.split()[1]
        elif line.startswith("Epochs:"):
            curr_res["epochs"] = line.split()[1]
        elif line.startswith("Batch:"):
            curr_res["batch"] = line.split()[1]
        elif line.startswith("Dataset:"):
            curr_res["dataset"] = line.split()[1]
        elif line.startswith("Loss:"):
            curr_res["loss_function"] = line.split()[1]
        elif line.startswith("Lr0:"):
            try:
                curr_res["lr0"] = float("0."+line.split()[1])
            except:
                curr_res["lr0"] = float(line.split()[1])
            
        if line.strip(" ").startswith("all"):
            P, R, mAP5, mAP95 = line.split()[3:]
            curr_res["all_P"] = P
            curr_res["all_R"] = R
            curr_res["all_F1"] = (float(P) * float(R)) / (float(P) + float(R))
            curr_res["all_mAP@.5"] = mAP5
            curr_res["all_mAP@.5:.95"] = mAP95

        elif line.strip(" ").startswith("knife"):
            P, R, mAP5, mAP95 = line.split()[3:]
            curr_res["knife_P"] = P
            curr_res["knife_R"] = R
            curr_res["knife_F1"] = (float(P) * float(R)) / (float(P) + float(R))
            curr_res["knife_mAP@.5"] = mAP5
            curr_res["knife_mAP@.5:.95"] = mAP95

        elif line.strip(" ").startswith("pistol") or line.strip(" ").startswith("gun"):
            P, R, mAP5, mAP95 = line.split()[3:]
            curr_res["pistol_P"] = P
            curr_res["pistol_R"] = R
            curr_res["pistol_F1"] = (float(P) * float(R)) / (float(P) + float(R))
            curr_res["pistol_mAP@.5"] = mAP5
            curr_res["pistol_mAP@.5:.95"] = mAP95

        elif line.strip(" ").startswith("Results saved to"):
            model_key = line.split("/")[-1].replace('\x1b[0m', '').replace('\n', '')
            model_info = model_key.split("_")

            curr_res["model_key"] = model_key

            results.append(curr_res)
            curr_res = dict()

    df = pd.DataFrame(results)
    df.set_index('model_key', inplace=True)
    return df


file_path = os.path.join(eval_results_dir, output_file)
df = process_file(file_path)
df.to_csv(f'{eval_results_dir}/output_csvs/{output_file_name}.csv')
df